# The Eikon API

## How to get Market past data with Python

In the previous espisode we saw how could we get data from a 'live feed', the Eikon server which sends information on a constant basis. We analyzed two methodologies to accomplish our final goal. In this way we can (in example) update the valutations of the positions we hold and calculate the key metrics of our portfolio every time an updated market price is retrieved. 

This information is very useful if we are building a system which controls our portfolio and generates insights, information day and night. The data retrieval and calculation can run persistently in the background and we can consult it every time we want by means of a more or less complex interface: a GUI, a Dashboard or a Web App. We will discuss this aspect in detail in the section regarding presenting data.

We can chose to send updates every X time, or at a give hour every day. We can even set thresholds on some key metrics, and send notifications if those limits are breached. But what about more complex metrics/indicators? 

Every asset manager knows that there is one piece of information that is missing and that is vital to perform more in-depht analysis: data from the past. This is a fundamental piece we need in order to be able to run practically any kind of financial application, apart from those which are purely based on the present situation of your portfolio. It is sufficient to think about 3 different, yet widely used indicators/metrics:

- moving averages: a mean of past prices.
- volatility: the standard deviation of past returns.
- maximum drawdown: basically the maximum loss over a time span.

All of them uses a series of values taken from a past time span: it can be one week, one year or a custom time frame. No matter the time, we must be able to obtain this information for our applications. This is why today's episode is based on how we can fetch past data using Eikon and its Python API.

## How past data is retrieved in Excel?

As we already saw in the introduction, the Eikon interface for Excel is accessible via an Excel COM Add-in during the Eikon terminal first installation and it is called Datastream. There are basically two ways of using the library:

1. Browsing the interface that appears on the Ribbon.
2. Typing formulas direcly in one cell.

The first method provides and 'easy' way to find the time series you need. It's called _DFO Navigator_ and it's accessed with the button _Find Series_. Clicking the button a GUI will pop-up and you can try to find the instruments you need with some filtered searches.

<img src="images/DSButtons.png">

I wrote easy in quotation marks because I always found the interface really legacy and hard to navigate. This is only my opinion but you can judge yourself looking at the example image below. Nowadays we are used to advanced search bars and engines and I found that this particular one would need a refresh. In the example I tried to search for the S&P500 and that is what I get as first attempt, whithout adding filters. Quite disappointing for a company that earns a quite good amount of money from their clients.

<img src="images/DFO_Navigator.png">

Once the instrument you are searching is found, the _Time Series Request_ Button will help you create the Data request.

<img src="images/DS_TSRequest.png" width="75%">

You can directly type the code in the corresponding field or make a search for it. You can easily customize your request with a lot of options that are accessed directly on the panel, and this is a very nice feature. 

The result is dumped on the excel spreadsheet in the first column. If you submit more requests at the same time more columns will be filled with data.

<img src="images/DS_TS.png">

If you pay attention to the image above, you will see that the add-in created a custom formula in the first cell: ```DSGRID```, with all the options that were specified during the manual request. Useless to say that if you know already the formula you can type it in one cell and you will still get your data. 

I will finish this Excel review by noticing just one thing: imagine we need to update the timeseries of all the S&P500 components: they are 500. How would you do it with this manual interface? Well, I will not answer the question because I would like you to come to the same conclusion as me: Read the next section.

## How to retrieve past data in Python

Let's go to the fun part now. As you probably know now, all the process saw in the previous section can be replicated using the Python Eikon API in a very similar way to what we saw for getting the live data: using a method of the API called ```get_timeseries```.

| Name | Type | Description | 
|----- |----- | ----------- |
|rics | string or list of strings | Single RIC or list of RICs to retrieve historical data for |
|start_date | string or datetime.datetime or datetime.timedelta | Starting date and time of the historical range. The string format is %Y-%m-%dT%H:%M:%S, for example, 2016-01-20T15:04:05. datetime.timedelta is negative number of day relative to datetime.now(). The default is datetime.now() + timedelta(-100). You can use the helper function get_date_from_today. See the Examples section for usage.|
|end_date | string or datetime.datetime or datetime.timedelta End date and time of the historical range. |
Possible string formats:
%Y-%m-%d, for example, 2017-01-20
%Y-%m-%dT%H:%M:%S, for example, 2017-01-20T15:04:05
datetime.timedelta is negative number of day relative to datetime.now().
The default is datetime.now().
You can use the helper function get_date_from_today. See the Examples section
for usage. |

interval string Data interval. Possible values are tick, minute, hour, daily, weekly, monthly,
quarterly, yearly.
The default is daily.
fields string or list of
strings
Use this parameter to filter the returned fields set. Available fields: TIMESTAMP,
VALUE, VOLUME, HIGH, LOW, OPEN, CLOSE, COUNT.
By default, all fields are returned.
count int, optional Maximum number of data points retrieved.
calendar string, optional Possible values: native, tradingdays, calendardays.
corax string, optional Possible values: adjusted, unadjusted

In [2]:
import eikon
import time
from datetime import datetime

# connect to the eikon api
eikon.set_app_key('07d8b517d50c495d9784b0bcad37c3f5f879e27f')

# start infinite loop get quote
# do something is there was no error
while True:
    # print current time
    print(f'{datetime.now().strftime("%d/%m/%Y %H:%M:%S")}: retrieving info')
    
    # get data from eikon
    df,e = eikon.get_data(['.SPX','AAPL.O'],['CF_LAST','CF_BID'])
    if e:
        print(f'the following error has been produced {e}')
        break
    else:
        # do what you want here 
        print(df)
    
    # wait ten seconds before querying again
    time.sleep(10) 
    break
    


03/02/2022 16:27:40: retrieving info
  Instrument  CF_LAST   CF_BID
0       .SPX   4536.6  4534.88
1     AAPL.O    175.6   175.58


In the code above: 
- We first setup the connection to Eikon, as we saw in the previous episodes.
- We start a an infinite loop usinge the ```while``` clause.
- We retrieve the information we need as usual.
- Using the module ```time``` we wait 10 seconds before repeating the process.

We can set a different value for the timeout to change the frequency as we wish. Here we put a ```break``` if the retrieval produces an error, but we can wrap any kind of logic that will stop our execution. This is the developer's choice!

One remark about this procedure: it is a static procedure. Meaning that there is a fixed dead-time between the start of the loop and the sleep interval. During this dead time, if the price of the instrument changes, we will not be able to catch the change. This is not ideal for all those applications that need quick in-time responses.

Ideally we would like to receive an update each time new data arrives to the server and perform our calculations asyncronously (i.e. process the data in a different/parallel thread), before receiveing new updates. Fortunately, this is achievable by using a built-in method of the Eikon API. Let's see how.

# A second, more efficient, approach

We will exploit a new method of the API called ```Streaming Prices```. The method is similar to ```get_data```, it accepts the same inputs (a list of instrument codes and a list of fields to retrieve) but in addition it supports event handling. This means that when a certain event happens (i.e.a new field is updated) the information contained in the event can be passed to a function and you can use is for your purposes. 

Let's see how this is translated in Python:


In [3]:
# define a callback function to print simple info
def print_update(streaming_price, instrument_name, fields):
    print("Update received for {}: {}".format(instrument_name, fields))
    return

# define the streaming price object
# use the callback to print the information
streaming_prices = eikon.StreamingPrices(
    instruments = ['.SPX', 'AAPL.O'], 
    fields = ['DSPLY_NAME', 'BID', 'ASK'],
    on_update = lambda streaming_price, instrument_name, fields :  
        print_update(streaming_price, instrument_name, fields)
)

# open streming
# wait 1 second
# close streaming
streaming_prices.open()
time.sleep(1)
streaming_prices.close()

Update received for AAPL.O: {'BID': 175.62, 'ASK': 175.64}


<StreamState.Closed: 1>

Update received for AAPL.O: {'BID': 175.64, 'ASK': 175.65}
Update received for .SPX: {'BID': 4535.64, 'ASK': 4538.9}
Update received for AAPL.O: {'BID': 175.63, 'ASK': 175.64}
Update received for AAPL.O: {'BID': 175.63, 'ASK': 175.64}


In the code above:
- we define a function that will print the information contained in the streming event.
- we define a streaming price with its inputs.
- we open the streaming, wait 1 second, and then close it.

As you can see the ```instruments``` and ```fields``` inputs are the same, but here we use the ```on_update``` event to define a callback to the ```print_update``` function, that will be called every time a new data arrives and will be passed the streaming information via the ```lambda``` function. If you want to know more about the functioning of ```lambda``` you can read this small tutorial [article](https://www.w3schools.com/python/python_lambda.asp). 

In the end we can achieve what we wanted in the beginning, a constant stream of data the is sent to us from the server, as we like: a nice waiter that serves data ready to be consumed, while we wait at the table and brew some coffee! ☕